# Context Manager

패턴에 잘 대응되기 때문에 유용하다.  
    
일반적으로 리소스 관리와 관련하여 컨텍스트 관리자를 자주 볼 수 있다.  
예를 들어, 일단 파일을 열면 파일 디스크립터 누수를 막기 위해 작업이 끝나면 적절히 닫히길 기대한다.  
또는 서비스나 소켓에 대한 연결을 열었을 때도, 적절하게 닫거나 임시 파일을 제거하는 등의 작업을 해야한다.  
     
- 사전조건과 사후조건을 가지고 있다. (주요 동작의 전후에 작업을 실행하려고 할 때, 유용하다)
- `__enter__`, `__exit__` 두개의 메소드로 구성된다.
- `__exit__`의 반환 값을 `True`로 하면, 잠재적으로 발생한 예외를 호출자에게 전파하지 않고, 멈춘다.
- `__exit__` 메소드 파라미터는 블록에서 발생한 예외를 파라미터로 받는다. (예외가 없으면 모두 `None`이다.)

## Resource manager without context manager

위의 모든 경우에 일반적으로 할당된 모든 리소스를 해제해야한다.  
모든 것이 잘 처리되었을 경우의 해제는 쉽지만 예외가 발생하거나, 오류를 처리해야 하는 경우는 어떻게 될까?  
가능한 모든 조합과 실행 경로를 처리하여 디버깅하는 것이 어렵다는 점을 감안할 때,  
이 문제를 해결하는 가장 일반적인 방법은 finally 블록에 정리 코드를 넣는 것이다.

```python
fd = open("filename.txt")
try:
    process_file(fd)
finally:
    fd.close()
```

이를 똑같은 방법이지만, 파이썬스럽게 코드를 변경하면 다음과 같다.

```python
with open(filename) as fd:
    process_file(fd)
```

## with (PEP-343)

- 컨텍스트 관리자로 진입하게 한다.
- 위의 경우 `open`함수는 컨텍스트 관리자 프로토콜을 구현한다.
- `with`는 `__enter__`를 호출한다.
- `__enter__` 메소드가 무엇을 반환하던 이를 `as` 뒤의 변수에 할당한다.
- `__enter__` 메소드가 특정한 값을 반환할 필요는 없다
- 메소드가 특정한 값을 반환하지 않는다면, 이를 `as`로 변수에 할당하지 않아도 된다.
- `with` 블록이 끝나면, 해당 컨텍스트 관리자 객체의 `__exit__` 메소드를 호출한다.

# Implement Context Manager

- `__enter__`, `__exit__` 메소드로 Context Manager를 구현할 수 있다.
    - 업무 도메인에 보다 얽히게 된다.
    - 책임이 커진다.
    - 하지 않아도 될 것들을 지원해야 한다.
- `contextlib`모듈을 이용해서 구현할 수 있다.
    - 기존 함수를 리팩토링하기 쉽다.
    - 어느 특정 객체에도 속하지 않은 컨텍스트 관리자가 필요한 경우 유용하다.
    - 많은 상태를 관리할 필요가 없다.
    - 다른 클래스와 독립되어 있는 컨텍스트 매니저 함수를 만들 수 있다.

## contextlib.contextmanger 데코레이터

- 해당 데코레이터를 적용하면, 해당 함수의 코드를 컨텍스트 관리자로 변환한다.
- 함수는 **제너레이터** 형태여야한다.
    - **제너레이터**는 문장을 `__enter__`와 `__exit__` 매직 매소드로 분리한다.
    
```python
import contextlib

@contextlib.contextmanger
def db_handler():
    # __enter__ 메소드 대응
    stop_database()
    yield
    # __exit__ 메소드 대응
    start_database()
    
with db_handler():
    db_backup()
```

## contextlib.ContextDecorator

컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스  

```python
class dbhandler_decorator(contextlib.Decorator):
    def __enter__(self):
        stop_database()
        
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()
        
@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")
```

- `with`문이 필요없다.
    - 함수를 호출하기만 하면, offline_backup함수가 컨텍스트 관리자 안에서 자동으로 실행된다.
- 완전히 독립적이다.
    - 데코레이터는 함수에 대해 아무것도 모르고, 그 반대도 마찬가지다.
- 단, 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없다.
- 재사용이 용이하다.

## contextlib.suppress
(*자세하게 공부해야함*)

컨텍스트 관리자에서 사용하는 util 패키지로 제공한 예외 중 하나가 발생한 경우에는 실패하지 않도록 한다.

- try/except 블록에서 코드를 실행하고 예외를 전달하거나 로그를 남기는 것은 비슷하다.
- suppress 메소드를 호출하면 로직에서 자체적으로 처리하고 있는 예외임을 명시한다.

```python
import contextlib

with contextlib.suppress(DataConversionException):
    parse_data(input_json_or_dict)
```